# Goal of Notebook

Create a transformer up to the code of 'Attention is all you need' for a case of giving it an input sequence and it would have to try to predict the output.<br>
Processes are per - timestep ( note : not per - token )

Done Task:
Positional Encoding

Current Task:
Embeddings of input(does it work for words and numbers?)

Next Task:
Attention (It is copied input, x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)))
Multi head Attention
Layer Norm

In [1]:
from torch import nn
import torch
import numpy as np
import math
import torch.nn.functional as f
import copy


/home/nkolln/mt/vis10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Input is a single list of tensors
#output is an array of size list by dimension
#This is constant?? Simpy add to layer
def positional_encoding(dim,max_len=1000):
    arr_pe = torch.zeros(max_len,dim)
    #Creates an array for the positions of rows
    arr_pos = torch.arange(0,max_len).unsqueeze(1)
    #Sine and cosine are applied an alternating dims so half are generated(Sin and cos apply to same subset basically to have the different outputs)
    div_term = torch.exp(torch.arange(0, dim, 2) * -(math.log(10000.0) / dim))
    #basically multipleis the 0-max len values by the div term which is unique per dim
    temp = arr_pos*div_term
    arr_pe[:,0::2]= torch.sin(temp)
    arr_pe[:,1::2]= torch.cos(temp)
    return(arr_pe.unsqueeze(0))

def scaled_dot_product_attention(q,k,v,dim):
    print(k)
    q_k = torch.matmul(q,k.T)
    q_k_scale = q_k * math.sqrt(dim)
    #can add a mask here

    q_k_scale = f.softmax(q_k_scale,dim=1)
    attn = torch.matmul(q_k_scale,v)
    return(attn)

def clones(component,N):
    return(nn.ModuleList([copy.deepcopy(component)for _ in range(0,N)]))


pe = positional_encoding(20)
    

In [51]:
#Embedding
#Embedding only works for discrete (Can we just set a ridiculously large embedding and treat continuous values as discrete)(Can we use convolutional kernels possibly?)
#Embedding embedds and then creates linear layers to make the v,k,q from same data,then process through scaled product attention, concat and apply final linear.
class EmbeddingLayer(nn.Module):
    def __init__(self,dic_size,dim,size):
        super().__init__()
        # self.dc = dic_size
        self.dim = dim
        self.embedding = nn.Embedding(dic_size,dim)
        self.pe = positional_encoding(dim,size)

    def forward(self,x):
        x = self.embedding(x) 
        #x = x* math.sqrt(self.dim)
        print(x.shape)
        #x = x + self.pe
        return x,pe



class AttentionHead(nn.Module):
    def __init__(self,l_layer):
        super().__init__()
        
        self.l1 = nn.Linear(l_layer,l_layer)
        self.l2 = nn.Linear(l_layer,l_layer)
        self.l3 = nn.Linear(l_layer,l_layer)
    def forward(self,x):
        q = f.relu(self.l1(x))
        k = f.relu(self.l2(x))
        v = f.relu(self.l3(x))
        return(x,q,k,v)

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super().__init__()


input = torch.LongTensor([[1,2,4],[4,3,2]])
data = input
dic_size = 1000;embed_size=512;attn_layer_size = embed_size
el = EmbeddingLayer(dic_size,embed_size,input.shape[1])
# print(type(data))
embed,pe = el(data)
print(embed)
attnhead = AttentionHead(attn_layer_size)
# print(attnhead(embed)[1])
# attnhead(embed)[1]

torch.Size([2, 3, 512])
tensor([[[-1.0572, -1.3308,  0.5802,  ..., -0.6959,  1.3029, -0.0801],
         [ 0.4477, -1.1618, -0.9530,  ...,  1.3338,  0.7754,  1.1468],
         [-1.7042,  0.7498,  0.5346,  ...,  0.1864, -0.6773, -1.8943]],

        [[-1.7042,  0.7498,  0.5346,  ...,  0.1864, -0.6773, -1.8943],
         [ 0.2036,  1.9969, -0.2026,  ..., -1.8149,  0.2619,  0.6535],
         [ 0.4477, -1.1618, -0.9530,  ...,  1.3338,  0.7754,  1.1468]]],
       grad_fn=<EmbeddingBackward0>)


# Testing

In [56]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        print(x.shape)
        print(self.pe.shape)
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

class TestClass(nn.Module):
    def __init__(self,position):
        super(TestClass,self).__init__()
        self.layer = nn.Sequential(nn.Embedding(1000, 512),position)
    
    def forward(self,x):
        return self.layer(x)


In [69]:
position = PositionalEncoding(512, 0)
test = TestClass(position)
print(input.shape)
out = test(data)
print(out.shape)
out

torch.Size([2, 3])
torch.Size([2, 3, 512])
torch.Size([1, 5000, 512])
torch.Size([2, 3, 512])


tensor([[[ 0.1705,  2.2181, -0.0212,  ...,  0.8106,  0.7591, -0.2599],
         [-0.0576,  1.6933,  1.4200,  ...,  1.1282, -1.1444,  1.1106],
         [ 2.3810,  0.7858,  0.2128,  ...,  1.8807,  0.8289,  2.0980]],

        [[ 1.4717,  2.2020, -0.7236,  ...,  1.8807,  0.8287,  2.0980],
         [ 3.0392, -0.1269,  1.9517,  ..., -0.2170, -0.0198,  1.2920],
         [ 0.0102,  0.7368,  1.5346,  ...,  1.1282, -1.1442,  1.1106]]],
       grad_fn=<AddBackward0>)

In [53]:
print(pe.shape)
pe[:, : 10].shape

torch.Size([1, 1000, 20])


torch.Size([1, 10, 20])

In [4]:
type(torch.exp(torch.arange(0, 20, 2, dtype=torch.float32) * -(math.log(10000.0) / 20)))

torch.Tensor

In [3]:
torch.arange(0, 20, 2, dtype=torch.float)

tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.])

In [2]:
V=11; batch_size = 30
data = torch.randint(1, V, size=(batch_size, 10))
data[:, 0] = 1
src = data.requires_grad_(False).clone().detach()
tgt = data.requires_grad_(False).clone().detach()

In [10]:
np.where(src!=tgt)

(array([], dtype=int64), array([], dtype=int64))

In [22]:
arr = torch.zeros(4,6)
tmp = (torch.arange(0,17,5).unsqueeze(1)*torch.arange(0,30,10))
print(tmp)
arr[:,0::2]=torch.sin(tmp)
arr[:,1::2]=torch.cos(tmp)
print(arr)

tensor([[  0,   0,   0],
        [  0,  50, 100],
        [  0, 100, 200],
        [  0, 150, 300]])
tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.0000,  1.0000, -0.2624,  0.9650, -0.5064,  0.8623],
        [ 0.0000,  1.0000, -0.5064,  0.8623, -0.8733,  0.4872],
        [ 0.0000,  1.0000, -0.7149,  0.6993, -0.9998, -0.0221]])


In [78]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(100, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4],[4,3,2]])
input = torch.randn(1)
input = input.long()
embedding(input)
print(embedding(input))



tensor([[0.8151, 1.3661, 1.1932]], grad_fn=<EmbeddingBackward0>)


In [42]:
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(5, 3)
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,2],[4,3,2,1]])
embedding(input)
print(embedding(input))



tensor([[[ 0.0326,  1.5697,  2.0568],
         [-0.0909, -0.0345, -0.1071],
         [ 0.0105, -0.3593,  0.5579],
         [-0.0909, -0.0345, -0.1071]],

        [[ 0.0105, -0.3593,  0.5579],
         [ 0.5077,  0.0521,  0.0648],
         [-0.0909, -0.0345, -0.1071],
         [ 0.0326,  1.5697,  2.0568]]], grad_fn=<EmbeddingBackward0>)


# Old Code

In [70]:
#Embedding
#Embedding only works for discrete (Can we just set a ridiculously large embedding and treat continuous values as discrete)(Can we use convolutional kernels possibly?)
#Embedding embedds and then creates linear layers to make the v,k,q from same data,then process through scaled product attention, concat and apply final linear.
class EmbeddingLayer(nn.Module):
    def __init__(self,dic_size,dim,size):
        super().__init__()
        # self.dc = dic_size
        self.dim = dim
        self.embedding = nn.Embedding(dic_size,dim)
        self.pe = positional_encoding(dim,size)

    def forward(self,x):
        x = self.embedding(x) 
        #x = x* math.sqrt(self.dim)
        print(x.shape)
        #x = x + self.pe
        return x,pe



class AttentionHead(nn.Module):
    def __init__(self,l_layer):
        super().__init__()
        
        self.l1 = nn.Linear(l_layer,l_layer)
        self.l2 = nn.Linear(l_layer,l_layer)
        self.l3 = nn.Linear(l_layer,l_layer)
    def forward(self,x):
        q = f.relu(self.l1(x))
        k = f.relu(self.l2(x))
        v = f.relu(self.l3(x))
        return(x,q,k,v)


input = torch.LongTensor([[1,2,4],[4,3,2]])
data = input
dic_size = 1000;embed_size=512;attn_layer_size = embed_size
el = EmbeddingLayer(dic_size,embed_size,input.shape[1])
# print(type(data))
embed,pe = el(data)
print(embed)
attnhead = AttentionHead(attn_layer_size)
# print(attnhead(embed)[1])
# attnhead(embed)[1]

torch.Size([2, 3, 512])
tensor([[[-0.0382,  0.7443, -0.7847,  ..., -0.3645, -1.9231, -0.4937],
         [ 1.9632,  0.8879,  0.8878,  ..., -0.4422,  0.0637,  0.5649],
         [ 0.0391,  0.8868,  0.8470,  ...,  0.4675, -1.5627,  1.2016]],

        [[ 0.0391,  0.8868,  0.8470,  ...,  0.4675, -1.5627,  1.2016],
         [ 1.6990,  1.3319, -0.3974,  ..., -1.2178,  0.6342, -0.0397],
         [ 1.9632,  0.8879,  0.8878,  ..., -0.4422,  0.0637,  0.5649]]],
       grad_fn=<EmbeddingBackward0>)
